In [1]:
import os
import json
import random
import requests
import random

from ipyleaflet import Map, GeoJSON

import ipyleaflet
import json
import pandas as pd
import os
import requests
from ipywidgets import link, FloatSlider, HTML
from branca.colormap import linear
from ipyleaflet import Choropleth, Map, Popup

## Two layer choropleth maps

Sections: 
1) Voting districts
2) Planning districts

Values:


### Voting districts

In [7]:
#load in voting district json
def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)
    
voting_districts_json = load_data(
    'https://opendata.arcgis.com/datasets/ca0f4261673541d798551f5cddc54bd6_0.geojson',
    'data_json',
     json.load) #json.loads turns geo_json into a dict
#voting_dis_json

#json formating
for d in voting_districts_json["features"]:
    d["PRECINCTID"] = d["properties"]["PRECINCTID"]
    
def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }
#add empty layer to map
voting_districts_map = Map(center=(29.9511, -90.0715), zoom=11)

voting_districts_layer = GeoJSON(
    data=voting_districts_json,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=random_color
)
voting_districts_map.add_layer(voting_districts_layer)

voting_districts_map

Map(center=[29.9511, -90.0715], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

### Planning districts

In [2]:
#https://gis.nola.gov/arcgis/rest/services/Planning/Planning_Districts/MapServer/0?f=pjson

#data loading function
def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)
    
planning_districts_json = load_data(
    'https://opendata.arcgis.com/datasets/878cc847e4b6412993c929138333f119_0.geojson',
    'orleans_planning_district.json',
     json.load) #json.loads turns geo_json into a dict
#planning_districts_json

In [12]:
def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }

planning_district_map = Map(center=(29.9511, -90.0715), zoom=11)

planning_district_layer = GeoJSON(
    data=planning_districts_json,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5, 
    },
    style_callback=random_color
)
planning_district_map.add_layer(planning_district_layer)

planning_district_map

Map(center=[29.9511, -90.0715], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

### Overlay the voting districts and planning districts layers

NOTE: the last added layer is the one that is highlighted on hover

In [23]:
#new map
overlay_map = Map(center=(29.9511, -90.0715), zoom=11)
overlay_map.add_layer(voting_districts_layer)
overlay_map.add_layer(planning_district_layer)
overlay_map

Map(center=[29.9511, -90.0715], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

### Remake chorpleth map with less lines

Voting districts - black outlines, red fill ----- Planning districts - grey outlines, blue fill

In [22]:
#new map
overlay_map2 = Map(center=(29.9511, -90.0715), zoom=11)
#planning district line layer
planning_district_layer2 = GeoJSON(
    data=planning_districts_json,
    style={
        'opacity': 1, 'dashArray': '0', 'fillOpacity': 0.0, 'weight': 1, 'color':'black'
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5, 'fillColor': 'red'
    })
   
overlay_map2.add_layer(planning_district_layer2)

#voting district line layer

voting_districts_layer2 = GeoJSON(
    data=voting_districts_json,
    style={
        'opacity': 1, 'dashArray': '0', 'fillOpacity': 0.0, 'weight': 1, 'color': 'grey'
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5, 'fillColor': 'blue'
    })
overlay_map2.add_layer(voting_districts_layer2)


overlay_map2

Map(center=[29.9511, -90.0715], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

### Remake choropleth map with overlapping colors

In [28]:
#new map
overlay_map3 = Map(center=(29.9511, -90.0715), zoom=11)
#planning district line layer
planning_district_layer2 = GeoJSON(
    data=planning_districts_json,
    style={
        'opacity': 1, 'dashArray': '0', 'fillOpacity': 0.1, 'weight': 1, 'color':'black', 'fillColor':'blue'
    })
   
overlay_map3.add_layer(planning_district_layer2)

#voting district line layer

voting_districts_layer2 = GeoJSON(
    data=voting_districts_json,
    style={
        'opacity': 1, 'dashArray': '0', 'fillOpacity': 0.1, 'weight': 1, 'color': 'grey', 'fillColor':'red'
    })
overlay_map3.add_layer(voting_districts_layer2)


overlay_map3

Map(center=[29.9511, -90.0715], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

### Add income color scale

In [34]:
#load in demographic data from csv file
with open('DECENNIALPL2020.P1_data_with_overlays_2021-10-05T211037.csv', 'r') as f:
    race = pd.read_csv(f, header=0)
#clean data
#total population
race['P1_001N'] = pd.to_numeric(race['P1_001N'],errors='coerce')
#number population black
race['P1_004N'] = pd.to_numeric(race['P1_004N'],errors='coerce')
#percentage black
race['%_Black'] = race['P1_004N']/race['P1_001N'] *100
race = race.dropna(subset=['%_Black'], axis = 0)
race = race.join(race['NAME'].str.split(',', 1, expand=True).rename(columns={0:'Precinct_ID', 1:'B'}))
#create dictionary: key = , value = 
mapping =  dict(zip(race['Precinct_ID'][0:352].tolist(), race['%_Black'].tolist()))
voting_district_race_layer = Choropleth(
                    geo_data=voting_districts_json,
                    choro_data=mapping,
                    colormap=linear.Reds_05,
                    #style={'fillOpacity': 1.0, "color":"black"},
                    key_on= 'PRECINCTID',
                    style={'opacity': 0.5, 'dashArray': '0', 'color': 'grey', 'fillOpacity': 0.5, 'weight': 1})
                    

color_map1 = Map(center=(29.9511, -90.0715), zoom=11)
color_map1.add_layer(voting_district_race_layer)
color_map1.add_layer(planning_district_layer2)
color_map1.layout.height="700px"
color_map1

Map(center=[29.9511, -90.0715], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…